#### diverse code snippets

In [ ]:
import gpe_module as gpe
import os
import glob
import pandas as pd
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("third_round_gRNA_no_gRNA_predicted_id.csv")

In [ ]:
df_full = pd.read_csv("protein_coding_gene_ids_ensembl_curated.csv")

In [ ]:
df_full = df_full.loc[:, ~df_full.columns.str.match("Unnamed")]

In [ ]:
third_round = df.merge(df_full, on="GENE NAME", how="inner")

In [ ]:
third_round = third_round.loc[:, ~third_round.columns.str.match("Unnamed")]

In [ ]:
third_round

In [ ]:
third_round.to_csv("forth_round_gRNA.csv")

In [ ]:
df = pd.read_csv("Summary_results/genes_with_four_gRNAs_predicted.csv")

In [ ]:
df = df.loc[:,~df.columns.str.match("Unnamed")]
df.head()

In [ ]:
df_primer = pd.read_csv("all_genes_four_gRNAs_primer_200_primer_designed.csv")

In [ ]:
df_primer = df_primer.loc[:,~df_primer.columns.str.match("Unnamed")]
df_primer

In [ ]:
df_merged = df.merge(df_primer, on="GRNA NAME", how="inner")

In [ ]:
df_merged = df_merged.loc[:,~df_merged.columns.str.match("GRNA CODING STRAND")]

In [ ]:
df_merged = df_merged.loc[:, ~df_merged.columns.str.match("PREDICTED GRNA_T")]

In [ ]:
df_merged = df_merged.loc[:, ~df_merged.columns.str.match("GENE SEQUENCES")]

In [ ]:
df_merged = df_merged.loc[:, ~df_merged.columns.str.match("SEQ SLICE")]

In [ ]:
df_merged

In [ ]:
df_merged.to_csv("_PCR_200_gRNA_primer_final.csv")

In [ ]:
guides_non_flat = [['ACUGCUUUAAAAAUUUUAGC', 'UACAUACAUCGUACUUUCCA'], 'ACAGCUGCUCACCUGGCACC', ['GGAAUCGGAGAAGGCCGCAA', 'CAGCGGCUCUUGCUCCGUUG', 'CGCAACGGAGCAAGAGCCGC', 'ACGGAGCAAGAGCCGCUGGA'], 'UGGACUCUUUUGGUUGUAGU', ['UUGAGGGGGAGAGCUCCGCC', 'ACUUGAGUUCUCAGUUGAGG', 'GACUUGAGUUCUCAGUUGAG', 'UGACUUGAGUUCUCAGUUGA'], ['CUCCAGGUCCAGCUCGCCAA', 'CCCGGCACAGCAGCCGCCCG', 'AGUAGCGCUCGUGCCGCACC'], 'AUGACUGAGCAAGUAGUUCA', 'GCGUGGGGAUCGUGUGGCAG', ['UGGGCGAAGUUGAAGUACUC', 'UGUGUGGAGUCGGCUGGAAG']]

In [ ]:
len(guides_non_flat)

In [ ]:
gene_names = ['ABCE1', 'ABCE1', 'ABHD17A', 'ABT1', 'ABT1', 'ABT1', 'ABT1', 'ACER2', 'ACKR1', 'ACKR1', 'ACKR1', 'ACKR1', 'ACOT1', 'ACOT1', 'ACOT1', 'ACOT2', 'ACSM2A', 'ACSM5', 'ACSM5']

In [ ]:
len(gene_names)

In [ ]:
guides_flat = [item for sublist in guides_non_flat for item in sublist]

In [ ]:
guides_flat

In [ ]:
gRNAs_synth_array_final = []
for i in range(len(guides_non_flat)):
    if len(guides_non_flat[i]) > 15:
        gRNAs_synth_array_final.append(guides_non_flat[i])
#     print(len(guides_non_flat[i]))
#     print(type(guides_non_flat[i]))
#     print(guides_non_flat[i])
    else:
        for item in guides_non_flat[i]:
            gRNAs_synth_array_final.append(item)

In [ ]:
gRNAs_synth_array_final

In [ ]:
import itertools

In [ ]:
merged = list(itertools.chain.from_iterable(guides_non_flat))

In [ ]:
merged

In [ ]:
df_primer_design, df_no_primer_design = gpe.primer3_primer_around_gRNA(df, upstream_dist=200, downstream_dist=200)

In [ ]:
df_primer_design.to_csv("all_genes_four_gRNAs_primer_200_primer_designed.csv")

In [ ]:
df_no_primer_design.to_csv("all_genes_four_gRNAs_primer_200_no_primer_designed.csv")

In [ ]:
name = '_gRNA_primer_final'
extension = 'csv'
all_filenames = [i for i in glob.glob(f"*{name}.{extension}")]

In [ ]:
all_filenames

In [ ]:
#combine all files in the list
df = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
#combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
df = df.loc[:,~df.columns.str.match("Unnamed")]

In [ ]:
len(df["GENE ID"].unique())

In [ ]:
#extract rows with at least one NaN entry
df_nan = df[df.isnull().any(axis=1)]

In [ ]:
df_nan.to_csv("gRNA_primer_missing.csv")

In [ ]:
df_nan_w_seq = gpe.extract_gene_seq_from_ens_id("gRNA_primer_missing.csv")

In [ ]:
df_full = df_nan.merge(df_nan_w_seq, on="GENE ID", how="inner")

In [ ]:
df_full = df_full[["GENE ID", "GENE NAME", "PREDICTED GRNA", "GENE SEQUENCES"]]

In [ ]:
df_full_loc = gpe.gRNA_hybridisation(df_full)

In [ ]:
df_full_loc

In [ ]:
def primer3_primer_around_gRNA(df_gRNAs_info, upstream_dist=250, downstream_dist=250):

    #list to hold sequences
    sequences_for_primer_design = []
    
    df_gRNAs_info["GRNA NAME"] = df_gRNAs_info["GENE NAME"] + '_loc_' + df_gRNAs_info["GRNA LOCATION"].astype(str)

    for row in range(len(df_gRNAs_info)):
        beginning = df_gRNAs_info["GRNA LOCATION"].iloc[row] - upstream_dist
        end = df_gRNAs_info["GRNA LOCATION"].iloc[row] + downstream_dist
        key = df_gRNAs_info["GRNA NAME"].iloc[row]
        value = df_gRNAs_info["GENE SEQUENCES"].iloc[row]
        value = str(value)
        seq_slice = value[beginning:end]
        sequences_for_primer_design.append(seq_slice)

    df_gRNAs_info['SEQ SLICE'] = sequences_for_primer_design
    
    primer_designed = {}
    primer_not_designed = {}
    
    for entry in range(len(df_gRNAs_info)):
        
        if len(df_gRNAs_info['SEQ SLICE'].iloc[entry]) == (upstream_dist+downstream_dist):
                
            seq_dict = {
            'SEQUENCE_ID': df_gRNAs_info['PREDICTED GRNA'].iloc[entry],
            'SEQUENCE_TEMPLATE': df_gRNAs_info['SEQ SLICE'].iloc[entry],
            }
            primer_designed[df_gRNAs_info['GRNA NAME'].iloc[entry]] = primer3.designPrimers(seq_dict,    
            {
                'PRIMER_OPT_SIZE': 20,
                'PRIMER_PICK_INTERNAL_OLIGO': 1,
                'PRIMER_INTERNAL_MAX_SELF_END': 8,
                'PRIMER_MIN_SIZE': 18,
                'PRIMER_MAX_SIZE': 25,
                'PRIMER_OPT_TM': 60.0,
                'PRIMER_MIN_TM': 57.0,
                'PRIMER_MAX_TM': 63.0,
                'PRIMER_MIN_GC': 20.0,
                'PRIMER_MAX_GC': 80.0,
                'PRIMER_MAX_POLY_X': 100,
                'PRIMER_INTERNAL_MAX_POLY_X': 100,
                'PRIMER_SALT_MONOVALENT': 50.0,
                'PRIMER_DNA_CONC': 50.0,
                'PRIMER_MAX_NS_ACCEPTED': 0,
                'PRIMER_MAX_SELF_ANY': 12,
                'PRIMER_MAX_SELF_END': 8,
                'PRIMER_PAIR_MAX_COMPL_ANY': 12,
                'PRIMER_PAIR_MAX_COMPL_END': 8,
                'PRIMER_PRODUCT_SIZE_RANGE': [[(upstream_dist+downstream_dist)-((upstream_dist+downstream_dist)/4), (upstream_dist+downstream_dist)]],
            })
            
        else:
            primer_not_designed[df_gRNAs_info['GRNA NAME'].iloc[entry]] = "Couldn't design primers," \
            "input gene sequence too short"
        
    primer_df = pd.DataFrame.from_dict(primer_designed, orient='index')
    primer_df["GRNA NAME"] = primer_df.index
    primer_df = primer_df[["GRNA NAME", "PRIMER_LEFT_0_SEQUENCE", "PRIMER_RIGHT_0_SEQUENCE", "PRIMER_PAIR_0_PRODUCT_SIZE", "PRIMER_LEFT_0", "PRIMER_RIGHT_0", "PRIMER_INTERNAL_0", "PRIMER_LEFT_0_TM", "PRIMER_RIGHT_0_TM"]]
    
    no_primer_df = pd.DataFrame.from_dict(primer_not_designed, orient='index')
    no_primer_df["GRNA NAME"] = no_primer_df.index
                                
    return primer_df, no_primer_df

In [ ]:
a, b = gpe.primer3_primer_around_gRNA(df_full_loc)

In [ ]:
df.to_csv("Summary_results/genes_four_gRNA_primer_final.csv")

In [ ]:
len_unique_entries = df["GENE ID"].value_counts().rename_axis('GENE ID').reset_index(name='counts')
len(len_unique_entries[len_unique_entries["counts"] != 4])

In [ ]:
df_full = pd.read_csv("Summary_results/genes_with_four_gRNAs_predicted.csv")

In [ ]:
#len_unique_entries_full = df["GENE ID"].value_counts().rename_axis('GENE ID').reset_index(name='counts')
#len(len_unique_entries_full[len_unique_entries_full["counts"] != 4])

In [ ]:
df_full = df_full.loc[:,~df_full.columns.str.match("Unnamed")]

In [ ]:
df_merged = df.merge(df_full, on="GRNA NAME", how="inner")

In [ ]:
df_merged

In [ ]:
df_merged.to_csv("all_genes_four_gRNAs_primer_")

In [ ]:
#new_df = pd.merge(df, df_full, on="GENE NAME", how="inner")

In [ ]:
#new_df.to_csv("second_round_gRNA.csv")

In [ ]:
#new_df

In [ ]:
#df.to_csv("genes_with_four_gRNAs_predicted.csv")

#### other possibly useful code snippets

In [ ]:
#df= pd.read_csv("genes_with_less_than_four_gRNAs_predicted_initially_gRNA_primer_final.csv")
# check if all unique entries are len 4 - if not take those and re-run
#df = pd.read_csv("AFDN_gRNA_primer_final.csv")
len_unique_entries = df["GENE ID"].value_counts().rename_axis('GENE ID').reset_index(name='counts')

In [ ]:
len(len_unique_entries[len_unique_entries["counts"] != 4])

In [ ]:
entries_length_below_four = len_unique_entries[len_unique_entries["counts"] != 4]

In [ ]:
entries_length_below_four

In [ ]:
genes_four_gRNA_lacking_primer = pd.merge(entries_length_below_four, df_full, on="GENE ID", how="inner")

In [ ]:
genes_four_gRNA_lacking_primer

In [ ]:
genes_four_gRNA_lacking_primer.to_csv("genes_four_gRNA_lacking_primer.csv")

In [ ]:
# check if entries in col null or empty
#df[df['column_name'] == ''].index
#pd.isna(df['column_name'])

In [ ]:
#https://www.biostars.org/p/487055/
from Bio import SeqIO
for re in SeqIO.parse('CHIKV1-X-gb_AB455493.fasta', 'fasta'):
    print('>{}\t{}'.format(str(re.description).replace('|', '\t'), re.seq))

In [ ]:
# split_fasta.py (assumes you have biopython installed, e.g. with pip install biopython)
# https://www.bramvandijk.com/blog/splitting-a-multi-fasta
import sys, math
from Bio import SeqIO

def batch_iterator(iterator, batch_size):
    """
    This is a generator function, and it returns lists of the
    entries from the supplied iterator.  Each list will have
    batch_size entries, although the final list may be shorter.
    (derived from https://biopython.org/wiki/Split_large_file)
    """
    entry = True  # Make sure we loop once
    while entry:
        batch = []
        while len(batch) < batch_size:
            try:
                entry = iterator.__next__()
            except StopIteration:
                entry = None
            if entry is None:
                break # EOF = end of file
            batch.append(entry)
        if batch:
            yield batch

if(len(sys.argv) != 3):
        sys.exit("usage: split_fasta.py MULTI_FASTA_FILE N_CHUNKS")

ffile=sys.argv[1]  # fasta file
chunks=sys.argv[2] # number of chunks
nseq = len([1 for line in open(ffile) if line.startswith(">")])
chunksize=math.ceil(nseq/int(chunks))
print("Splitting multi-fasta file of", nseq, "sequences into chunks of size", chunksize)

records = SeqIO.parse(open(ffile), "fasta")
for i, batch in enumerate(batch_iterator(records, chunksize)):
        filename = "chunk_%i.fasta" % (i + 1)
        with open(filename, "w") as handle:
                count = SeqIO.write(batch, handle, "fasta")
        print("Wrote %i sequences to %s" % (count, filename))

sys.exit("Done.")